# Head

## imports

In [1]:
# -*- coding: utf-8 -*-

In [19]:

import os
import sys
import numpy as np
import glob
import math
import random
from tqdm import tqdm_notebook as tqdm
import shutil
import keras

from config import config

## Tuning

In [3]:
#Tuning
dataset = config.DATASET
n_points = config.N_POINTS
cell = config.CELL
num_classes = config.NUM_CLASSES
extension = config.EXTENSION
class_name = config.CLASS_NAME

In [4]:
train_or_test = ["train","test"]

In [5]:
#読み込み，書き込み用ディレクトリ
data_dir = config.DATA_DIR
vox_dir = config.VOX_DIR
fig_dir =  config.FIG_DIR
dist_dir = config.DIST_DIR

In [6]:
dir_list = [data_dir , vox_dir , fig_dir , dist_dir]
for directory in dir_list:
    if os.path.exists(directory) == False:
        os.makedirs(directory)

In [7]:
# TrainとTestのデータdirectory作成
for t in train_or_test:
    for cl in class_name:
        if os.path.exists(data_dir + cl + "/" + t + "/") == False:
            os.makedirs(data_dir + cl + "/" + t + "/")

# Function

In [8]:
def load_off(filename):
    # read OFF file
    with open(filename,"r") as handle:
        off = handle.read().rstrip().split("\n")
        
    #OFFファイルが不正かどうか判定
    if off[0] != "OFF":
#         print("{} is broken!!".format(filename))
        params = list(off[0].split(" "))
        n_vertices = int(params[0].strip("OFF"))
        n_faces = int(params[1])
        off.insert(0, "OFF")
    
    else:
        #get params and faces
        params = list(map(int, off[1].split(" ")))
        n_vertices = params[0]
        n_faces = params[1]

    # read  Vertex coordinates
    vertices = []
    for n in range(n_vertices):
        coords = list(map(float, off[2+n].split()))
        vertices.append(coords)

    # read information of faces
    faces = []
    for n in range(n_faces):
        connects = list(map(int, off[2 + n_vertices + n].split(" ")[1:4]))
        faces.append(connects)

    return vertices, faces

In [9]:
def load_stl(filename):
    # read STL file
    with open(filename,"r") as handle:
        stl = handle.read().rstrip().split("\n")
        
    #get vertice
    vertice = []
    for i in range(len(stl)):
        pool = stl[i].split()
        if pool[0] == "vertex":
            vertex = list(map(float, pool[1:]))
            vertice.append(vertex)
            
    seen = []
    unique_vertice = [x for x in vertice if x not in seen and not seen.append(x)]
            
    #get faces
    faces = []
    for n in range(len(stl)):
        if stl[n].split() == ['outer', 'loop']:
            indexes = []
            for i in range(3):
                index = get_index_2d_list(unique_vertice, list(map(float,stl[n+i+1].split()[1:])))
                indexes.append(index)
            faces.append(indexes)

    return unique_vertice, faces

In [10]:
def generate_points(points, face_ids, num_generated_point=1000):
    
    points = np.array(points)
    face_ids = np.array(face_ids)
    
    # compute area
    vec1 = points[face_ids[:,0]] - points[face_ids[:,1]]
    vec2 = points[face_ids[:,1]] - points[face_ids[:,2]]
    cross_product = np.cross(vec1, vec2)
    area = np.sqrt(np.power(cross_product, 2).sum(axis=1))

    # cumsum area
    cum_area = np.cumsum(area)

    # generate random 
    random = np.random.rand(num_generated_point) * cum_area[-1]

    # convert random to index
    #random_idx = [bisect.bisect_left(cum_area, x) for x in random]
    random_idx = np.random.choice(np.arange(len(face_ids)), size=num_generated_point, p=area/cum_area[-1])

    #import pdb; pdb.set_trace()

    # generate points
    r1 = np.tile(np.random.rand(num_generated_point), (3, 1)).T
    r2 = np.tile(np.random.rand(num_generated_point), (3, 1)).T

    generated_points = (1 - np.sqrt(r1))*points[face_ids[random_idx, 0]] + np.sqrt(r1)*(1-r2)*points[face_ids[random_idx, 1]] + np.sqrt(r1)*r2*points[face_ids[random_idx, 2]]

    return generated_points, random_idx

In [11]:
def voxilize(np_pc,cell):
# ボクセル化した配列を返す
    max_dist = 0.0
    for it in range(0,3):
        # 最大値と最小値の距離を求める
        min_ = np.amin(np_pc[:,it])
        max_ = np.amax(np_pc[:,it])
        dist = max_-min_

        #xyzで一番並行距離が大きいのを求める
        if dist > max_dist:
            max_dist = dist
            
    for it in range(0,3):

        # 最大値と最小値の距離を求める
        min_ = np.amin(np_pc[:,it])
        max_ = np.amax(np_pc[:,it])
        dist = max_-min_
        
        #中心座標を 0,0,0にセットする（原点が中心にくるようにする）
        np_pc[:,it] = np_pc[:,it] - dist/2 - min_

        #covered cell
        cls = cell - 3

        #ボクセル一個当たりのサイズを求める
        vox_sz = max_dist/(cls-1)

        #上で算出した値で各点を割る。これで各点は(-14, 14)の範囲の値になる
        np_pc[:,it] = np_pc[:,it]/vox_sz

        #各点が全て正の整数になるよう移動。これで各点は[0, 30]になる（多分）
        np_pc[:,it] = np_pc[:,it] + (cls-1)/2


    #整数にする
    np_pc = np.rint(np_pc).astype(np.uint32)


    #３０＊３０＊３０の配列を作り，点が存在する場合は1、存在しない場合は0を入力する。
    vox = np.zeros([cell-2,cell-2,cell-2])

    # (pc_x, pc_y, pc_z)にnp_pcの座標を代入する
    for (pc_x, pc_y, pc_z) in np_pc:

#     # 点が存在しても20%の確率で0とし、データにノイズを加え、汎用性を上げている
#     # ここ，ノイジーなデータの場合８０よりも小さい数字にしたほうがいいかもね？
#         if random.randint(0,100) < 80:
        vox[pc_x, pc_y, pc_z] = 1

    np_vox = np.zeros([1,cell,cell,cell,1])
    np_vox[0, 1:-1, 1:-1, 1:-1,0] = vox

    return np_vox

In [12]:
def voxel_scatter(np_vox):
# キレイに整形するやつ
    #空の配列を作る
    vox_scat = np.zeros([0,3], dtype= np.uint32)

    #32回
    for x in range(0,np_vox.shape[1]):
        #32回
        for y in range(0,np_vox.shape[2]):
            #32回
            for z in range(0,np_vox.shape[3]):
                #（ｘ，ｙ，ｚ）に１が入っていればその座標を返す
                if np_vox[0,x,y,z,0] == 1.0:
                    arr_tmp = np.zeros([1,3],dtype=np.uint32)
                    arr_tmp[0,:] = (x,y,z)
                    vox_scat = np.concatenate((vox_scat,arr_tmp))
    return vox_scat

In [13]:
def load_vox_stl(filename, n_points = n_points):
    #shape(N,1,32,32,32)の配列を返す
    vertices, faces = load_stl(filename)
    points,faces = generate_points(vertices, faces, n_points)
    # ValueError: sequence too large; cannot be greater than 32の回避策
    # list 2 numpy.ndarray
    pc = np.empty((len(points), len(points[0])))
    pc[:] = points
    vox = voxilize(pc,cell)

    return vox

In [14]:
def load_vox_off(filename, n_points = n_points):
    #shape(N,1,32,32,32)の配列を返す
    vertices, faces = load_off(filename)
    points,faces = generate_points(vertices, faces, n_points)
    # ValueError: sequence too large; cannot be greater than 32の回避策
    # list 2 numpy.ndarray
    pc = np.empty((len(points), len(points[0])))
    pc[:] = points
    vox = voxilize(pc,cell)

    return vox

In [15]:
# XYZ 長さの取得
def calc_dist(filename):
    vertices, faces = load_stl(filename)
    point =  np.array(vertices)
    dist = np.zeros(3)
    for i in range(0,3):
        min_ = np.amin(point[:,i])
        max_ = np.amax(point[:,i])
        dist_ = max_ - min_
        if dist_ < 0.001:
            dist_ = 0.001
        dist[i] = dist_
    return dist

# ディレクトリ操作

In [16]:

if dataset == "3D_SAMPLE":
    # train_or_testの分ける
    for cl in class_name:
        namelist = glob.glob(data_dir+ cl +"/*")
        namelist.remove(data_dir+ cl + "/train")
        namelist.remove(data_dir+ cl + "/test")
        for i in range(len(namelist)):
            if i%5 == 0 :
                shutil.move(namelist[i], data_dir + cl + "/test/.")
    #             print(namelist[i])
            else:
                shutil.move(namelist[i], data_dir + cl + "/train/.")
    #             print(namelist[i])
#  ====================rename===============================
for cl in class_name:
    print('Renameing {} files ...'.format(cl))
    for t in train_or_test:
        i = 0
        #ファイル名を取得
        namelist = glob.glob(data_dir+ cl +"/" + t + "/*")
        if data_dir+ cl +"/" + t + "/0" + extension in namelist:
            print(t +" is already renamed!!")
            continue
        for file in namelist:
            os.rename(file, data_dir+ cl +'/' + t +"/"+ str(i) + extension)
            i+=1
print("Rename all!")
dir_rename_flag = True
#  ====================rename===============================

Renameing bathtub files ...
train is already renamed!!
test is already renamed!!
Renameing chair files ...
train is already renamed!!
test is already renamed!!
Renameing dresser files ...
train is already renamed!!
test is already renamed!!
Renameing night_stand files ...
train is already renamed!!
test is already renamed!!
Renameing table files ...
train is already renamed!!
test is already renamed!!
Renameing bed files ...
train is already renamed!!
test is already renamed!!
Renameing desk files ...
train is already renamed!!
test is already renamed!!
Renameing monitor files ...
train is already renamed!!
test is already renamed!!
Renameing sofa files ...
train is already renamed!!
test is already renamed!!
Renameing toilet files ...
train is already renamed!!
test is already renamed!!
Rename all!


# ボクセルのnumpy保存

In [17]:
off_voxilize = False

for t in train_or_test:
    for cl in  tqdm(class_name):
        print(cl)
        
        #すでにボクセル化が済んていればスキップ
        if os.path.exists(vox_dir + "x_train.npy") == True:
#             print("{} is exist.".format(vox_dir + cl + t + ".npy"))
            off_voxilize = True
            continue
        
        #すでに存在してるnpyファイルはスキップ
        if os.path.exists(vox_dir + cl + t + ".npy") == True:
            print("{} is exist.".format(data_dir + cl + t + ".npy"))
            continue
        
        num = glob.glob(data_dir+ cl +"/" + t + "/*")
        for i in tqdm(range(len(num))):
#             print(data_dir + cl + "/" + t +"/"+ str(i) + ".stl")
            if i ==0  :
                vox_data = load_vox_off(data_dir + cl + "/" + t +"/"+ str(i) + extension)
                continue
            vox_data_ =  load_vox_off(data_dir + cl + "/" + t +"/"+ str(i) + extension)
            
            vox_data = np.append(vox_data, vox_data_, axis=0)
        np.save(vox_dir + cl + t + ".npy", vox_data)
#クラス毎，TestTrain毎ボクセル化されたnpzが生成される（N,32,32,32,1）

bathtub
ModelNet10/bathtubtrain.npy is exist.
chair
ModelNet10/chairtrain.npy is exist.
dresser
ModelNet10/dressertrain.npy is exist.
night_stand
ModelNet10/night_standtrain.npy is exist.
table
ModelNet10/tabletrain.npy is exist.
bed
ModelNet10/bedtrain.npy is exist.
desk
ModelNet10/desktrain.npy is exist.
monitor
ModelNet10/monitortrain.npy is exist.
sofa
ModelNet10/sofatrain.npy is exist.
toilet
ModelNet10/toilettrain.npy is exist.



bathtub
ModelNet10/bathtubtest.npy is exist.
chair
ModelNet10/chairtest.npy is exist.
dresser
ModelNet10/dressertest.npy is exist.
night_stand
ModelNet10/night_standtest.npy is exist.
table
ModelNet10/tabletest.npy is exist.
bed
ModelNet10/bedtest.npy is exist.
desk
ModelNet10/desktest.npy is exist.
monitor
ModelNet10/monitortest.npy is exist.
sofa
ModelNet10/sofatest.npy is exist.
toilet
ModelNet10/toilettest.npy is exist.



In [20]:

#ココらへん可読性わるすぎ
if not off_voxilize:
    # データ整形
    for t in train_or_test:
        swich_npy = True 
        num_of_data = []#init
        for cl in class_name:
            if swich_npy == True:
                npy = np.load(vox_dir  + cl + t + ".npy")
                num_of_data.append(npy.shape[0])
                swich_npy = False
            else:
                npy_ = np.load(vox_dir  + cl + t + ".npy")
                npy = np.append(npy,npy_,axis=0)
                num_of_data.append(npy_.shape[0])
        if t == "train":
            x_train = npy
            y_train = num_of_data
        if t == "test":
            x_test = npy
            y_test = num_of_data
            
        
    # ラベルの生成
    for t in train_or_test:
        if t == "train":
            for i in range(num_classes):
                if i == 0:
                    label = np.full(y_train[i] , i )    
                else:
                    label_ = np.full(y_train[i] , i )   
                    label = np.append(label, label_, axis=0)
            y_train = label

        if t == "test":
            for i in range(num_classes):
                if i == 0:
                    label = np.full(y_test[i] , i )    
                else:
                    label_ = np.full(y_test[i] , i )   
                    label = np.append(label, label_, axis=0)
            y_test = label


# convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    # 保存
    npy = ["x_train.npy" , "x_test.npy" , "y_train.npy" , "y_test.npy"]
    data = [x_train , x_test , y_train , y_test]
    for i in range(len(npy)):
        np.save(vox_dir + npy[i],data[i])
        
    # 不要ファイルの除去
    for i in os.listdir(vox_dir):
        if not i in npy:
            os.remove(vox_dir + i)

# 距離の計算

In [21]:
dist_calcrated = False
for t in train_or_test:
    for cl in  tqdm(class_name):
        print(cl)
        
        #すでに距離の計算が済んていればスキップ
        if os.path.exists(dist_dir + "x_train.npy") == True:
#             print("{} is exist.".format(vox_dir + cl + t + ".npy"))
            dist_calcrated = True
            continue
        
        #すでに存在してるnpyファイルはスキップ
        if os.path.exists(dist_dir + cl + t + ".npy") == True:
#             print("{} is exist.".format(data_dir + cl + t + ".npy"))
            continue
        
        num = glob.glob(data_dir+ cl +"/" + t + "/*")
        for i in tqdm(range(len(num))):
            if i ==0  :
                dist = calc_dist(data_dir + cl + "/" + t +"/"+ str(i) + extension)
                dist = [dist]
                continue
            dist_ =  calc_dist(data_dir + cl + "/" + t +"/"+ str(i) + extension                                                                                                              )
            
            dist = np.append(dist, [dist_], axis=0)
#             print(dist.shape)
        np.save(dist_dir + cl + t + ".npy", dist)

bathtub
chair
dresser
night_stand
table
bed
desk
monitor
sofa
toilet



bathtub
chair
dresser
night_stand
table
bed
desk
monitor
sofa
toilet



In [22]:
if not dist_calcrated:
    # データ整形
    for t in train_or_test:
        swich_npy = True 
        num_of_data = []#init
        for cl in class_name:
            if swich_npy == True:
                npy = np.load(dist_dir  + cl + t + ".npy")
                num_of_data.append(npy.shape[0])
                swich_npy = False
            else:
                npy_ = np.load(dist_dir  + cl + t + ".npy")
                npy = np.append(npy,npy_,axis=0)
                num_of_data.append(npy_.shape[0])
        if t == "train":
            x_train = npy
            y_train = num_of_data
        if t == "test":
            x_test = npy
            y_test = num_of_data
            
        
    # ラベルの生成
    for t in train_or_test:
        if t == "train":
            for i in range(num_classes):
                if i == 0:
                    label = np.full(y_train[i] , i )    
                else:
                    label_ = np.full(y_train[i] , i )   
                    label = np.append(label, label_, axis=0)
            y_train = label

        if t == "test":
            for i in range(num_classes):
                if i == 0:
                    label = np.full(y_test[i] , i )    
                else:
                    label_ = np.full(y_test[i] , i )   
                    label = np.append(label, label_, axis=0)
            y_test = label


# convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    # 保存
    npy = ["x_train.npy" , "x_test.npy" , "y_train.npy" , "y_test.npy"]
    data = [x_train , x_test , y_train , y_test]
    for i in range(len(npy)):
        np.save(dist_dir + npy[i],data[i])
        
    # 不要ファイルの除去
#     for i in os.listdir(dist_dir):
#         if not i in npy:
#             os.remove(dist_dir + i)